## Preparación dataset

In [14]:
### Cargamos librería 
!pip install --upgrade xlrd 
!pip install yfinance 
import pandas as pd 
import numpy as np # numpy y pandas to data wrangling 
from datetime import datetime, timedelta # to work w date
import yfinance as yf # to import some others data as cryptocurrencies or stocks
from functools import partial, reduce # to transform variable dic in a dataframe  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Google Trends

Google trends es una herramienta creada en 2006 por [***Google***](https://support.google.com/trends/answer/6248105?hl=es&ref_topic=6248052) que permite monitorear la evolución del número de búsquedas realizadas para una determinada palabra clave o tema a lo largo del tiempo. Todo ello basado en la propia base de datos de Google que almacena todas las búsquedas que los usuarios realizan diariamente. La herramienta permite consultar resultados de búsquedas por país o región (en tiempo real, frecuencia horaria, diaria, semanal o mensual), categoría (finanzas, tecnología, etc.) y tipo de búsqueda (web, imágenes, noticias, compras o búsqueda en YouTube). 

Siguiendo a [***Google***](https://support.google.com/trends/answer/4365533?hl=es), las tendencias de búsquedas se presentan a través de un indicador que se construye a partir del siguiente proceso de normalización:
-  Cada punto de datos se divide por el total de búsquedas de la región geográfica y el intervalo de tiempo de interés para comparar su popularidad relativa. 
- A continuación, los números resultantes se escalan a un intervalo del 0 al 100 en función de la proporción de un tema con respecto al total de búsquedas sobre todos los temas.

Por lo tanto, las regiones que registran el mismo interés de búsqueda de un término no siempre tienen los mismos volúmenes de búsquedas totales.

La forma en que se construyen los indicadores, y algunas cuestiones relacionadas a las diferentes frecuencias de tiempo, lleva a considerar algunas cuestiones particulares en los datos a la hora de importar las series para todo el período de interés. En [***1_1_googleTrends***](https://colab.research.google.com/drive/16EsslAqhCxdLE7faten7jimdueDAYsXB#scrollTo=N5B37d9opaMa) se explican esos detalles para construir el *dataframe* **gtrends**. No obstante, en esta sección importamos directamente la base previamente creada. 

In [15]:
aux = 'https://docs.google.com/spreadsheets/d/1tHq6j9qVaNOoEEk5FQPpCAMZ6hP2Bn6C/edit?usp=sharing&ouid=105868423796285576163&rtpof=true&sd=true'
link ='https://drive.google.com/uc?id=' + aux.split('/')[-2]
gtrends = pd.read_excel(link)
gtrends = gtrends.set_index("Date")
gtrends = gtrends.loc[:, ['adjusted' in i for i in gtrends.columns]]

In [16]:
gtrends

,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,interest rate_adjusted,fed_adjusted,...,inflation_adjusted,taxes_adjusted,digital_wallet_top_adjusted,covid19_adjusted,pandemic_adjusted,elon musk_adjusted,do kwon_adjusted,future_commodities_adjusted,banking_adjusted,foreign_currency_adjusted
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-01,0.0,0.63,3.00,0.71,0.00,0.00,1.45,8.64,16.38,6.37,...,11.04,29.68,15.30,0.00,0.45,0.60,7.0,27.00,63.48,26.00
2015-08-02,0.0,0.64,2.31,0.53,0.00,0.00,1.45,9.76,13.86,5.88,...,12.96,25.97,9.86,0.00,0.70,0.66,6.3,22.95,49.68,24.44
2015-08-03,0.0,0.72,2.19,0.44,0.00,0.00,2.90,10.24,23.10,8.33,...,18.24,41.87,11.39,0.00,0.44,0.60,5.9,32.40,92.00,34.84
2015-08-04,0.0,0.90,2.16,0.46,0.00,0.00,2.61,9.28,20.16,9.31,...,17.52,43.99,13.09,0.00,0.60,0.63,8.5,31.95,87.40,34.84
2015-08-05,0.0,0.79,1.92,0.37,0.00,0.00,2.61,9.60,21.84,9.31,...,16.32,42.93,11.22,0.00,0.56,0.78,8.6,31.95,85.56,34.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,61.00,10.07,...,100.00,32.34,34.30,4.48,3.92,2.00,0.9,50.05,71.44,85.50
2022-06-11,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,53.00,6.89,...,53.00,23.10,34.30,4.06,3.29,2.00,0.9,38.50,50.16,65.70
2022-06-12,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,46.00,7.42,...,46.00,21.78,30.80,3.92,3.71,2.40,0.9,35.42,41.80,59.40


Como se puede ver, el *dataset* *gtrends* presenta las tendencias de búsquedas diarias de los términos y temas que consideramos relevantes para predecir el precio de la criptomoneda ethereum. Cada palabra o tópico presenta tres indicadores: la base diaria, histórica y diaria ajustada. Sin embargo, para el ejercicio de predicción utilizaremos solamente la base diaria ajustada de cada término. Los detalles sobre los tres tipos de indicadores y el motivo de utilización de la serie ajustada también se explican en [***1_1_googleTrends***](https://colab.research.google.com/drive/16EsslAqhCxdLE7faten7jimdueDAYsXB#scrollTo=N5B37d9opaMa).     

### Criptomonedas 

El siguiente paso es construir la base de criptomonedas. Por un lado, importamos información de ethereum, nuestro target. Por otro lado, también obtenemos información de bitcoin, los cambios relacionados a la cotización de esta pueden estar correlacionados con los cambios en ethereum, por lo que también la incluimos a la lista de predictores. 

[***CryptoDataDownload***](https://www.cryptodatadownload.com/) es una plataforma que brinda, entre otras cosas, información histórica de la cotización de diferentes criptomonedas a partir de la API Poloniex. Entre ellas, se encuentran las cotizaciones de ethereum y bitcoin. Los datos comprenden el precio de apertura y clausura en un momento del tiempo dado (horario, diario, etc.), el precio más alto y bajo, y el volumen de transacciones.

In [17]:
# ### Ethereum
# variable importada de Poloniex Data (incluye valores anteriores a 2017)
ethereum1 = pd.read_csv('https://www.cryptodatadownload.com/cdd/Poloniex_ETHUSDT_d.csv', skiprows=1)
# ethereum1.rename(columns = 'Volume')
ethereum1 = ethereum1.set_index('date')
ethereum1 = ethereum1.sort_index()
ethereum1.index
ethereum1.index = pd.to_datetime(ethereum1.index)
ethereum1.index = [d.date() for d in ethereum1.index]
ethereum1.index.names = ['Date']
ethereum1 = ethereum1.rename(columns={col: col+'_eth' for col in ethereum1.columns})
# promedio cierre y apertura precio ethereum
ethereum1['eth_close_open_mean'] = ethereum1.loc[:,['close_eth',	'open_eth']].mean(axis=1)
# agregamos a la variable el rezago de un día de la variable a predecir  
for i in range(1, 8):
  ethereum1[f'y_lag{i}'] = ethereum1['eth_close_open_mean'].shift(-i)

### Bitcoin
bitcoin = pd.read_csv('https://www.cryptodatadownload.com/cdd/Poloniex_BTCUSDT_d.csv', skiprows=1)
bitcoin = bitcoin.set_index('date')
bitcoin = bitcoin.sort_index()
bitcoin.index = pd.to_datetime(bitcoin.index)
bitcoin.index = [d.date() for d in bitcoin.index]
bitcoin.index.names = ['Date']
bitcoin = bitcoin.rename(columns={col: col+'_btc' for col in bitcoin.columns})

In [18]:
## Unión criptomonedas
crypto_pol = pd.merge(ethereum1, bitcoin, left_index=True, right_index=True)
del [ethereum1, bitcoin]
crypto_pol = crypto_pol.drop(['symbol_eth','symbol_btc'], axis = 1) 
crypto_pol.index = pd.to_datetime(crypto_pol.index)

In [19]:
crypto_pol

,unix_eth,open_eth,high_eth,low_eth,close_eth,Volume ETH_eth,Volume USDT_eth,eth_close_open_mean,y_lag1,y_lag2,...,y_lag5,y_lag6,y_lag7,unix_btc,open_btc,high_btc,low_btc,close_btc,Volume BTC_btc,Volume USDT_btc
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-08,1438992000,1.650000,1.850000,0.500000,1.400000,2.740000e+02,176.540000,1.525000,1.250000,1.240000,...,1.825,1.310,1.645,1438992000,275.920000,282.240000,262.650000,263.000000,1.032690e+03,3.801000
2015-08-11,1439251200,1.400000,1.400000,0.550000,1.100000,2.429800e+02,251.520000,1.250000,1.240000,1.765000,...,1.310,1.645,1.520,1439251200,266.220000,269.260000,261.610000,269.260000,2.474900e+02,0.943500
2015-08-12,1439337600,1.100000,1.400000,0.900000,1.380000,4.768100e+02,404.340000,1.240000,1.765000,2.150000,...,1.645,1.520,1.410,1439337600,269.260000,275.630000,263.330000,265.410000,6.821900e+02,2.558000
2015-08-13,1439424000,1.380000,2.150000,1.300000,2.150000,5.862800e+02,347.840000,1.765000,2.150000,1.825000,...,1.520,1.410,1.515,1439424000,265.410000,269.880000,260.000000,263.000000,8.389900e+02,3.189000
2015-08-14,1439510400,2.150000,2.150000,2.140000,2.150000,5.348000e+01,24.920000,2.150000,1.825000,1.310000,...,1.410,1.515,1.615,1439510400,263.000000,265.000000,255.000000,265.000000,5.562300e+02,2.129000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-21,1658361600,1521.451055,1605.000000,1465.387047,1575.676862,7.156517e+06,4682.421737,1548.563959,1555.724628,1542.233655,...,NaN,NaN,NaN,1658361600,23215.188719,23418.820243,22351.900000,23149.677861,4.184934e+06,182.833809
2022-07-22,1658448000,1575.872814,1645.030459,1517.719293,1535.576441,1.189203e+07,7500.311293,1555.724628,1542.233655,1573.477193,...,NaN,NaN,NaN,1658448000,23146.815304,23737.085142,22515.733046,22682.066631,4.510801e+06,194.966919
2022-07-23,1658534400,1535.389663,1596.377181,1489.591155,1549.077646,7.230743e+06,4702.563475,1542.233655,1573.477193,1576.601465,...,NaN,NaN,NaN,1658534400,22678.513618,22991.367833,21960.675489,22456.493172,5.161666e+06,231.804017


### Unión inputs 

In [20]:
# unión con merge
df = pd.merge(gtrends, crypto_pol, left_index=True, right_index=True)
# reordenar columnas, poniendo el target adelante
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
# visualización
df

,Volume USDT_btc,cryptocurrency_adjusted,crypto_adjusted,bitcoin_adjusted,bitcoin price_adjusted,ethereum_adjusted,ethereum price_adjusted,stock market_adjusted,wall street_adjusted,interest rate_adjusted,...,y_lag4,y_lag5,y_lag6,y_lag7,unix_btc,open_btc,high_btc,low_btc,close_btc,Volume BTC_btc
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-08,3.801000,0.0,0.48,1.74,0.27,0.00,0.00,1.45,8.32,16.38,...,2.150000,1.825000,1.310000,1.645000,1438992000,275.920000,282.240000,262.650000,263.000000,1.032690e+03
2015-08-11,0.943500,0.0,0.88,1.89,0.49,0.00,0.00,2.61,9.92,19.74,...,1.825000,1.310000,1.645000,1.520000,1439251200,266.220000,269.260000,261.610000,269.260000,2.474900e+02
2015-08-12,2.558000,0.0,0.87,1.83,0.47,0.00,0.00,3.48,10.40,20.58,...,1.310000,1.645000,1.520000,1.410000,1439337600,269.260000,275.630000,263.330000,265.410000,6.821900e+02
2015-08-13,3.189000,0.0,0.87,1.95,0.35,0.00,0.00,2.61,10.40,18.48,...,1.645000,1.520000,1.410000,1.515000,1439424000,265.410000,269.880000,260.000000,263.000000,8.389900e+02
2015-08-14,2.129000,0.0,0.83,1.68,0.32,0.00,0.00,2.32,10.88,16.80,...,1.520000,1.410000,1.515000,1.615000,1439510400,263.000000,265.000000,255.000000,265.000000,5.562300e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,247.612626,5.0,6.46,8.41,8.40,9.10,7.56,18.00,6.48,61.00,...,1208.954979,1222.789134,1152.640234,1077.132870,1654819200,30093.109097,30363.965398,28863.371181,29091.044620,7.291809e+06
2022-06-11,177.801516,5.0,7.14,8.99,9.80,15.34,16.74,8.70,5.94,53.00,...,1222.789134,1152.640234,1077.132870,1041.031988,1654905600,29091.044620,29428.819243,28120.500000,28408.993958,5.092090e+06
2022-06-12,263.103277,7.2,8.16,11.02,11.20,15.34,15.39,7.20,6.03,46.00,...,1152.640234,1077.132870,1041.031988,1061.923798,1654992000,28423.198276,28528.145588,26578.603357,26600.000000,7.240280e+06


In [21]:
# pasamos todas las variables a numéricas
df = df.apply(pd.to_numeric)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2499 entries, 2015-08-08 to 2022-06-14
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Volume USDT_btc              2499 non-null   float64
 1   cryptocurrency_adjusted      2499 non-null   float64
 2   crypto_adjusted              2499 non-null   float64
 3   bitcoin_adjusted             2499 non-null   float64
 4   bitcoin price_adjusted       2499 non-null   float64
 5   ethereum_adjusted            2499 non-null   float64
 6   ethereum price_adjusted      2499 non-null   float64
 7   stock market_adjusted        2499 non-null   float64
 8   wall street_adjusted         2499 non-null   float64
 9   interest rate_adjusted       2499 non-null   float64
 10  fed_adjusted                 2499 non-null   float64
 11  bankruptcy_adjusted          2499 non-null   float64
 12  china_adjusted               2499 non-null   float64
 13  

In [22]:
# guardar el dataset en carpeta data set del proyecto en google drive
from google.colab import drive
drive.mount('drive')
df.to_csv('input.csv')
!cp input.csv "/content/drive/MyDrive/ds/proyecto/dataset/"

Mounted at drive
